This stores SQL Queries for our reference.

In [ ]:
-- 1. Drop existing tables to avoid conflicts and ensure correct vector dimensions
DROP TABLE IF EXISTS conversation_logs;
DROP TABLE IF EXISTS user_memories;

-- 2. Enable vector extension (just in case)
create extension if not exists vector;

-- 3. Create table for Semantic Logs (Gemini = 768 dimensions)
create table conversation_logs (
  id bigint generated by default as identity primary key,
  session_id text not null,
  role text not null,
  content text not null,
  embedding vector(768),  -- <--- CRITICAL: Must be 768 for Gemini
  created_at timestamp with time zone default timezone('utc'::text, now())
);

-- 4. Create table for Structured Facts
create table user_memories (
  id bigint generated by default as identity primary key,
  session_id text not null,
  memory_type text,
  content jsonb,
  confidence float,
  last_accessed timestamp,
  embedding vector(768)   -- <--- CRITICAL: Must be 768 for Gemini
);

-- 5. Create the search function
create or replace function match_conversation(
  query_embedding vector(768), 
  match_threshold float, 
  match_count int,
  p_session_id text
)
returns table (
  id bigint,
  content text,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    conversation_logs.id,
    conversation_logs.content,
    1 - (conversation_logs.embedding <=> query_embedding) as similarity
  from conversation_logs
  where 1 - (conversation_logs.embedding <=> query_embedding) > match_threshold
  and session_id = p_session_id
  order by conversation_logs.embedding <=> query_embedding
  limit match_count;
end;
$$;

In [ ]:
create or replace function match_memories(
  query_embedding vector(768), 
  match_threshold float, 
  match_count int,
  p_session_id text
)
returns table (
  id bigint,
  memory_type text,
  content jsonb,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    user_memories.id,
    user_memories.memory_type,
    user_memories.content,
    1 - (user_memories.embedding <=> query_embedding) as similarity
  from user_memories
  where 1 - (user_memories.embedding <=> query_embedding) > match_threshold
  and session_id = p_session_id
  order by user_memories.embedding <=> query_embedding
  limit match_count;
end;
$$;

In [ ]:
-- 1. Explicitly drop the old function first
DROP FUNCTION IF EXISTS match_memories(vector, float8, int, text);

-- 2. Ensure the table has the column
ALTER TABLE user_memories 
ADD COLUMN IF NOT EXISTS created_at TIMESTAMPTZ DEFAULT NOW();

-- 3. Create the new version with the timestamp
CREATE OR REPLACE FUNCTION match_memories(
  query_embedding vector(768), 
  match_threshold float8, 
  match_count int,
  p_session_id text
)
RETURNS TABLE (
  id bigint,
  memory_type text,
  content jsonb,
  created_at TIMESTAMPTZ, -- The new return column
  similarity float8
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  SELECT
    user_memories.id,
    user_memories.memory_type,
    user_memories.content,
    user_memories.created_at,
    1 - (user_memories.embedding <=> query_embedding) AS similarity
  FROM user_memories
  WHERE 1 - (user_memories.embedding <=> query_embedding) > match_threshold
  AND user_memories.session_id = p_session_id
  ORDER BY user_memories.embedding <=> query_embedding
  LIMIT match_count;
END;
$$;

In [ ]:
-- 1. Drop the function so we can change its signature
DROP FUNCTION IF EXISTS match_memories(vector, float8, int, text);

-- 2. Force the table column to be TIMESTAMPTZ (with timezone)
-- If there is data, this will safely convert it to UTC
ALTER TABLE user_memories 
ALTER COLUMN created_at TYPE TIMESTAMPTZ 
USING created_at AT TIME ZONE 'UTC';

-- 3. Recreate the function with the exact matching return type
CREATE OR REPLACE FUNCTION match_memories(
  query_embedding vector(768), 
  match_threshold float8, 
  match_count int,
  p_session_id text
)
RETURNS TABLE (
  id bigint,
  memory_type text,
  content jsonb,
  created_at TIMESTAMPTZ, -- <--- Must match the table exactly
  similarity float8
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  SELECT
    user_memories.id,
    user_memories.memory_type,
    user_memories.content,
    user_memories.created_at,
    1 - (user_memories.embedding <=> query_embedding) AS similarity
  FROM user_memories
  WHERE 1 - (user_memories.embedding <=> query_embedding) > match_threshold
  AND user_memories.session_id = p_session_id
  ORDER BY user_memories.embedding <=> query_embedding
  LIMIT match_count;
END;
$$;